# Imports.

In [ ]:
import pandas as pd
import numpy as np
import scipy

from IPython import get_ipython

import os


from comp_econ import comp_econ as ce


# Problem 1.

## Define A and b per the Miranda and Fackler textbook.

In [ ]:
A = np.matrix([
    [54, 14, -11, 2],
    [14, 50, -4, 29],
    [-11, -4, 55, 22],
    [2, 29, 22, 95]
])
print("Matrix A:")
print(A)

b = np.matrix(np.ones(shape=(A.shape[0], 1)))
print("\nVector b:")
print(b)



## 1a. Solve for x using LU Decomposition.

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html

print("Running LU Decomposition method.")
P, L, U = scipy.linalg.lu(A)
assert np.allclose(A, (P @ L @ U)), "Check LU decomposition."

# Ax = b
# A = LU, or very close to A.
# --> LUx = b
# --> x = (LU)^-1 * b
LU_inv = scipy.linalg.inv((L @ U))
x_lu = (LU_inv @ b).round(4)
print("Vector x_lu:")
print(x_lu)

## 1b. and 1c. Solve for x using both Gauss-Jacobi and Gauss-Seidel methods.

In [ ]:
print("Running Gauss-Jacobi method.")
x_gj, num_iter_gj = ce.gjacobi(A, b, np.zeros(shape=(4, 1)))
x_gj = x_gj.round(4)
print("Vector x_gj:")
print(x_gj)

print("\nRunning Gauss-Seidel method.")
x_gs, num_iter_gs = ce.gseidel(A, b, np.zeros(shape=(4, 1)))
x_gs = x_gs.round(4)
print("Vector x_gs:")
print(x_gs)

## Check the answers of all 3 methods.

In [ ]:
# x_lu ~ x_gj and x_gj ~ x_gs --> x_lu ~ x_gs, by transitivity.
assert np.allclose(x_lu, x_gj) and np.allclose(x_gj, x_gs), "Solution vectors should all be approximately equal."

# Problem 2. The magic command %timeit includes more information than MATLAB's tic tok.

## Define A, A_inv, and b for problem 2.

In [ ]:
# https://numpy.org/doc/stable/reference/random/generated/numpy.random.standard_normal.html
# https://numpy.org/doc/stable/reference/random/index.html#random-quick-start

rng = np.random.default_rng()
A = np.matrix(
    rng.standard_normal(size=(100, 100))
)
A_inv = np.linalg.inv(A)
b = np.matrix(
    rng.standard_normal(size=(100, 1))
)

# Per the question instructions, perform runs of 1, 10, and 50. By default, %timeit performs 7 runs.
runs_list = [1, 10, 50]

## 2a. x = A\b is equivelent to np.linalg.solve().

In [ ]:
print("Solving for x using np.linalg.solve().")
for run_num in runs_list:
    get_ipython().run_line_magic("timeit", f"-n 1000 -r {run_num} x = np.linalg.solve(a=A, b=b)")
# Assign the result to x_solve to indicate it was found via np.linalg.solve().
x_solve = np.linalg.solve(a=A, b=b)
# Per the documention, check the solution.
assert np.allclose(np.dot(A, x_solve), b), "x_solve is not close to b."
print("Run times for x = A\\b in have been printed.")

## 2b. PLU decomposition.

In [ ]:
print("Solving for x using PLU decomposition.")
P, L, U = scipy.linalg.lu(A)
PLU_inv = scipy.linalg.inv((P @ L @ U))
for run_num in runs_list:
    get_ipython().run_line_magic("timeit", f"-n 1000 -r {run_num} x_plu = (PLU_inv @ b)")
# Assign the result to x_plu to indicate it was found via PLU decomposition.
x_plu = (PLU_inv @ b)
assert np.allclose(A, (P @ L @ U)), "Issue with PLU decomposition."
assert np.allclose(np.dot(A, x_plu), b), "x_plu is not close to b."
print("Run times for PLU decomposition have been printed.")

## 2c. x = A_inv @ b

In [ ]:
print("Solving for x using A_inv @ b.")
for run_num in runs_list:
    get_ipython().run_line_magic("timeit", f"-n 1000 -r {run_num} x_ainv = A_inv @ b")
# Assign the result to x_ainv to indicate it was found via A_inv @ b.
x_ainv = A_inv @ b
assert np.allclose(np.dot(A, x_ainv), b), "x_ainv is not close to b."
print("Run times for A_inv @ b have been printed.")

## Check the answers of all 3 methods.

In [ ]:
assert np.allclose(x_solve, x_plu) and np.allclose(x_plu, x_ainv), "Solution vectors should all be approximately equal."

# Problem 3.

## 3a. Looking at the demand equation, the parameter of interest is b since it is multiplied by the log of the price p. 

In [ ]:
image_dir = os.getenv("ECON_560_IMAGE_DIR")